# Import et etudes fichiers

## import

In [180]:
# !pip install gspread
# # Connection à notre google drive pour écrire et importer de la données de nos google sheets par la suite
# from google.colab import auth
# import gspread
# from google.auth import default
# #autenticating to google
# auth.authenticate_user()
# creds, _ = default()
# gc = gspread.authorize(creds)
# from google.colab import files

# installations et imports des autres packages
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [181]:
# # Augmenter la limite de débit de données IOPub
# from IPython.display import display, HTML
# display(HTML("<script>Jupyter.notebook.kernel.execute('NotebookApp.iopub_data_rate_limit = 10000000')</script>"))

In [182]:
# # Avant d'utiliser df.tail(), configurez les options d'affichage pour afficher toutes les colonnes
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Fonctions

In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

def etude_fichier(df):
    print("Nombre de colonnes :", df.shape)
    print()
    print("Le type est : \n", df.dtypes)
    print()
    print('Nombre de valeurs uniques :')
    print(df.nunique())
    print()
    print('Le nombre de valeurs manquantes :\n', df.isnull().sum())

def analyse_statistique(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    stats = df.describe(include='all')

    stats.loc['median'] = df[numeric_cols].median()
    stats.loc['skewness'] = df[numeric_cols].skew()
    stats.loc['kurtosis'] = df[numeric_cols].kurtosis()

    return stats

def identifier_differences_entre_cles(df1, df2, keys_df1=None, keys_df2=None):
    if keys_df1 is None:
        keys_df1 = [df1.columns[0]]
    if keys_df2 is None:
        keys_df2 = [df2.columns[0]]

    unique_keys_df1 = set(df1[keys_df1].drop_duplicates().apply(tuple, axis=1))
    unique_keys_df2 = set(df2[keys_df2].drop_duplicates().apply(tuple, axis=1))

    keys_only_in_df1 = unique_keys_df1 - unique_keys_df2
    keys_only_in_df2 = unique_keys_df2 - unique_keys_df1

    print("Nombre de clés absentes dans DF1:", len(keys_only_in_df1))
    print("Nombre de clés absentes dans DF2:", len(keys_only_in_df2))

def etudier_jointures(df1, df2):
    id_columns_df1 = [col for col in df1.columns if 'id' in col.lower()]
    id_columns_df2 = [col for col in df2.columns if 'id' in col.lower()]

    results = {}

    for on_column_df1 in id_columns_df1:
        for on_column_df2 in id_columns_df2:
            inner_unique_keys = len(pd.merge(df1, df2, how='inner', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))
            left_unique_keys = len(pd.merge(df1, df2, how='left', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))
            right_unique_keys = len(pd.merge(df1, df2, how='right', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))
            outer_unique_keys = len(pd.merge(df1, df2, how='outer', left_on=on_column_df1, right_on=on_column_df2).drop_duplicates(subset=on_column_df1))

            results[f"{on_column_df1} - {on_column_df2}"] = {
                'inner': inner_unique_keys,
                'left': left_unique_keys,
                'right': right_unique_keys,
                'outer': outer_unique_keys
            }

    for key, values in results.items():
        print(f"Colonnes de jointure : {key}")
        print("Nombre de clés uniques après jointure interne:", values['inner'])
        print("Nombre de clés uniques après jointure à gauche:", values['left'])
        print("Nombre de clés uniques après jointure à droite:", values['right'])
        print("Nombre de clés uniques après jointure externe:", values['outer'])
        print()

def traiter_valeurs_manquantes(df, method='mean', columns=None):
    if method == 'mean':
        if columns is None:
            return df.fillna(df.mean())
        else:
            return df.fillna(df.mean()[columns])
    elif method == 'median':
        if columns is None:
            return df.fillna(df.median())
        else:
            return df.fillna(df.median()[columns])
    elif method == 'mode':
        if columns is None:
            return df.fillna(df.mode().iloc[0])
        else:
            return df.fillna(df.mode().iloc[0][columns])
    else:
        return df.fillna(method=method)

def detecter_outliers(df, column, seuil=2.0):
    if column not in df.columns:
        raise ValueError(f"La colonne '{column}' n'existe pas dans le DataFrame.")

    series = df[column]
    z_scores = np.abs((series - series.mean()) / series.std())
    outliers_mask = z_scores > seuil
    outliers_percentage = (outliers_mask.mean() * 100).round(2)

    print(f"Le pourcentage de valeurs considérées comme des outliers en utilisant le Z-score au seuil 2 dans la colonne '{column}' est {outliers_percentage}%")

def plot_skewness_kurtosis(data, column):
    skewness = data[column].skew()
    kurtosis = data[column].kurtosis()

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    sns.histplot(data[column], kde=True)
    plt.axvline(x=data[column].mean(), color='r', linestyle='dashed', linewidth=2)
    plt.title(f'Skewness: {skewness:.2f}')

    plt.subplot(1, 2, 2)
    sns.histplot(data[column], kde=True)
    plt.axvline(x=data[column].mean(), color='r', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() + data[column].std(), color='g', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() - data[column].std(), color='g', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() + 2 * data[column].std(), color='b', linestyle='dashed', linewidth=2)
    plt.axvline(x=data[column].mean() - 2 * data[column].std(), color='b', linestyle='dashed', linewidth=2)
    plt.title(f'Kurtosis: {kurtosis:.2f}')

    plt.tight_layout()

    if skewness > 0:
        skewness_analysis = "La distribution est inclinée positivement vers la droite (queue à droite)."
    elif skewness < 0:
        skewness_analysis = "La distribution est inclinée positivement vers la gauche (queue à gauche)."
    else:
        skewness_analysis = "La distribution est parfaitement symétrique."

    if kurtosis > 0:
        kurtosis_analysis = "La distribution est leptokurtique, avec des pics plus fins et des queues plus épaisses."
    elif kurtosis < 0:
        kurtosis_analysis = "La distribution est platykurtique, avec des pics plus larges et des queues plus minces."
    else:
        kurtosis_analysis = "La distribution est mésokurtique, similaire à une distribution normale."

    print("Analyse de la répartition (skewness):", skewness_analysis)
    print("Analyse de l'aplatissement (kurtosis):", kurtosis_analysis)

    plt.show()

def correlation_matrix(df):
    corr_matrix = df.corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Matrice de Corrélation")
    plt.show()

def visualiser_valeurs_manquantes(df):
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
    plt.title("Visualisation des Valeurs Manquantes")
    plt.show()

def encoder_variables_categorielles(df, columns=None):
    if columns is None:
        columns = df.select_dtypes(include='object').columns
    return pd.get_dummies(df, columns=columns, drop_first=True)

def reduire_dimension_pca(df, n_components=None):
    pca = PCA(n_components=n_components)
    components = pca.fit_transform(df.select_dtypes(include=[np.number]))
    columns = [f"PC{i+1}" for i in range(components.shape[1])]
    df_pca = pd.DataFrame(components, columns=columns)
    return pd.concat([df.drop(columns=df.select_dtypes(include=[np.number]).columns), df_pca], axis=1)


## Sheet data


In [184]:
df_data = pd.read_csv('C:\\Users\\antoi\\OneDrive\\Bureau\\OC drive\\OC\\P9\\DATA\\DisponibiliteAlimentaire_2017.csv')
# # Ouvrez la feuille de calcul
# sheet_url = 'https://docs.google.com/spreadsheets/d/1Rfr4o1Fv47qUU2opVdOWbYnuxc4M-5x3YE6uBjTCVSY/edit#gid=2088935678'
# worksheet = gc.open_by_url(sheet_url).sheet1

# # Obtenez les données sous forme de liste de listes
# data = worksheet.get_all_values()

# # Transformez les données en un DataFrame
# df_data = pd.DataFrame(data)

# # Vous pouvez également définir la première ligne comme en-têtes de colonne
# df_data.columns = df_data.iloc[0]
# df_data = df_data[1:]

# # Affichez les 5 premières lignes du DataFrame
# df_data.tail()

In [185]:
etude_fichier(df_data)

Nombre de colonnes : (176600, 14)

Le type est : 
 Code Domaine               object
Domaine                    object
Code zone                   int64
Zone                       object
Code Élément                int64
Élément                    object
Code Produit                int64
Produit                    object
Code année                  int64
Année                       int64
Unité                      object
Valeur                    float64
Symbole                    object
Description du Symbole     object
dtype: object

Nombre de valeurs uniques :
Code Domaine                 1
Domaine                      1
Code zone                  174
Zone                       174
Code Élément                17
Élément                     17
Code Produit                98
Produit                     98
Code année                   1
Année                        1
Unité                        4
Valeur                    7250
Symbole                      2
Description du Symbole     

**Unités**

Unité = 'Milliers de tonnes', 'kg', Kcal/personne/jour', 'g/personne/jour'

Description du Symbole = 'Données standardisées', 'Donnée calculée'

In [186]:
# Sélection des colonnes
colonnes_a_supprimer = ['Code Domaine', 'Code zone','Description du Symbole', 'Code Élément', 'Code Produit',  'Code année', 'Symbole', "Domaine"]

# Supprimer les colonnes du dataframe final
df_data = df_data.drop(columns=colonnes_a_supprimer)

# Afficher les premières lignes du nouveau dataframe
df_data.head()

,Zone,Élément,Produit,Année,Unité,Valeur
0,Afghanistan,Production,Blé et produits,2017,Milliers de tonnes,4281.0
1,Afghanistan,Importations - Quantité,Blé et produits,2017,Milliers de tonnes,2302.0
2,Afghanistan,Variation de stock,Blé et produits,2017,Milliers de tonnes,-119.0
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,Milliers de tonnes,0.0
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,Milliers de tonnes,6701.0


## sheet population

In [187]:
df_population = pd.read_csv("C:\\Users\\antoi\\OneDrive\\Bureau\\OC drive\\OC\\P9\\DATA\\Population_2000_2018.csv")
# # Ouvrez la feuille de calcul
# sheet_url = 'https://docs.google.com/spreadsheets/d/1p8doV6_TWwk1O5X5ElHmpqkp7P_Pz_XaJywvEdUqPHU/edit#gid=260060951'
# worksheet = gc.open_by_url(sheet_url).sheet1

# # Obtenez les données sous forme de liste de listes
# data = worksheet.get_all_values()

# # Transformez les données en un DataFrame
# df_population = pd.DataFrame(data)

# # Vous pouvez également définir la première ligne comme en-têtes de colonne
# df_population.columns = df_population.iloc[0]
# df_population = df_population[1:]

# # Affichez les 5 premières lignes du DataFrame
# df_population.tail()

In [188]:
etude_fichier(df_population)

Nombre de colonnes : (4411, 15)

Le type est : 
 Code Domaine               object
Domaine                    object
Code zone                   int64
Zone                       object
Code Élément                int64
Élément                    object
Code Produit                int64
Produit                    object
Code année                  int64
Année                       int64
Unité                      object
Valeur                    float64
Symbole                    object
Description du Symbole     object
Note                       object
dtype: object

Nombre de valeurs uniques :
Code Domaine                 1
Domaine                      1
Code zone                  238
Zone                       238
Code Élément                 1
Élément                      1
Code Produit                 1
Produit                      1
Code année                  19
Année                       19
Unité                        1
Valeur                    4398
Symbole                   

In [189]:
# Liste des colonnes à supprimer
colonnes_a_supprimer = ['Code Domaine', 'Code zone', 'Code Élément', 'Élément','Code Produit', 'Code année', 'Unité', 'Symbole', 'Description du Symbole', 'Note', "Domaine", "Produit"]

# Supprimer les colonnes du dataframe final
df_population = df_population.drop(columns=colonnes_a_supprimer)

# Afficher les premières lignes du nouveau dataframe
df_population.head()

,Zone,Année,Valeur
0,Afghanistan,2000,20779.953
1,Afghanistan,2001,21606.988
2,Afghanistan,2002,22600.770
3,Afghanistan,2003,23680.871
4,Afghanistan,2004,24726.684


In [190]:
# Renommer la colonne
df_population = df_population.rename(columns={"Valeur": "Population"})

In [191]:
# Convertion en int
df_population['Population'] = pd.to_numeric(df_population['Population'], errors='coerce')

# Convertion en integer
df_population['Population'] = df_population['Population'].astype(int)

# Multiplier par1000
df_population['Population'] = df_population['Population']*1000

df_population.head()

,Zone,Année,Population
0,Afghanistan,2000,20779000
1,Afghanistan,2001,21606000
2,Afghanistan,2002,22600000
3,Afghanistan,2003,23680000
4,Afghanistan,2004,24726000


# Jointures

In [192]:
# Jointures des deux df
data = pd.merge(df_data, df_population, on=['Zone', 'Année'])
data.head()

,Zone,Élément,Produit,Année,Unité,Valeur,Population
0,Afghanistan,Production,Blé et produits,2017,Milliers de tonnes,4281.0,36296000
1,Afghanistan,Importations - Quantité,Blé et produits,2017,Milliers de tonnes,2302.0,36296000
2,Afghanistan,Variation de stock,Blé et produits,2017,Milliers de tonnes,-119.0,36296000
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,Milliers de tonnes,0.0,36296000
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,Milliers de tonnes,6701.0,36296000


In [193]:
etude_fichier(data)

Nombre de colonnes : (176600, 7)

Le type est : 
 Zone           object
Élément        object
Produit        object
Année           int64
Unité          object
Valeur        float64
Population      int32
dtype: object

Nombre de valeurs uniques :
Zone           174
Élément         17
Produit         98
Année            1
Unité            4
Valeur        7250
Population     173
dtype: int64

Le nombre de valeurs manquantes :
 Zone          0
Élément       0
Produit       0
Année         0
Unité         0
Valeur        0
Population    0
dtype: int64


# Manipulation des données

In [194]:
# ragarder quelles sont les éléments et le nombre de lignes
unique_elements_with_counts_elements = data['Élément'].value_counts()
print(unique_elements_with_counts_elements)

Disponibilité intérieure                                         15905
Importations - Quantité                                          15260
Disponibilité alimentaire en quantité (kg/personne/an)           14618
Disponibilité de matière grasse en quantité (g/personne/jour)    14512
Disponibilité de protéines en quantité (g/personne/jour)         14507
Nourriture                                                       14498
Disponibilité alimentaire (Kcal/personne/jour)                   14476
Résidus                                                          12567
Exportations - Quantité                                          12113
Variation de stock                                               11299
Production                                                       10334
Pertes                                                            5813
Alimentation pour touristes                                       5560
Autres utilisations (non alimentaire)                             5292
Alimen

In [195]:
# Regadrer quelles sont les unités et le nombre de lignes
unique_elements_with_counts_products = data['Produit'].value_counts()
print(unique_elements_with_counts_products)

Maïs et produits                   2593
Blé et produits                    2581
Pommes de Terre et produits        2486
Riz et produits                    2452
Lait - Excl Beurre                 2395
Oeufs                              2347
Légumineuses Autres et produits    2336
Orge et produits                   2301
Soja                               2222
Céréales, Autres                   2206
Légumes, Autres                    2204
Fruits, Autres                     2198
Arachides Decortiquees             2197
Edulcorants Autres                 2194
Haricots                           2141
Sucre Eq Brut                      2137
Graisses Animales Crue             2137
Feve de Cacao et produits          2088
Huil Plantes Oleif Autr            2083
Plantes Oleiferes, Autre           2077
Viande de Bovins                   2065
Viande de Volailles                2061
Avoine                             2060
Coco (Incl Coprah)                 2051
Tomates et produits                2046


In [196]:
# Regadrer quelles sont les unités et le nombre de lignes
unique_elements_with_counts_units = data['Unité'].value_counts()
print(unique_elements_with_counts_units)

Milliers de tonnes    118487
g/personne/jour        29019
kg                     14618
Kcal/personne/jour     14476
Name: Unité, dtype: int64


Modificatyion pour avoir les résultats dans des unités cohérentes

In [197]:
# ajout d'une colonne pour faire les moidifcations d'unités
data['Nombres de jour'] = 365
print(data.shape)

(176600, 8)


In [198]:
# Identifier les lignes où l'unité est "Milliers de tonnes"
is_milliers_de_tonnes = data['Unité'] == "Milliers de tonnes"

# Convertir les valeurs en kilogrammes pour ces lignes
data.loc[is_milliers_de_tonnes, 'Valeur'] *= 1_000_000

# Mettre à jour l'unité pour ces lignes
data.loc[is_milliers_de_tonnes, 'Unité'] = 'kg'

In [199]:
# Identifier les lignes où l'unité est "g/personne/jour"
is_g_per_person_per_day = data['Unité'] == "g/personne/jour"

# Effectuer le calcul en tenant compte de la population et du nombre de jours
data.loc[is_g_per_person_per_day, 'Valeur'] *= data.loc[is_g_per_person_per_day, 'Population'] * data.loc[is_g_per_person_per_day, 'Nombres de jour']

# Convertir les valeurs en kilogrammes pour ces lignes
data.loc[is_g_per_person_per_day, 'Valeur'] *=0.001  # Convertir les grammes en kilogrammes

# Mettre à jour l'unité pour ces lignes
data.loc[is_g_per_person_per_day, 'Unité'] = 'kg'

In [200]:
# Identifier les lignes où l'unité est "Kcal/personne/jour"
is_kcal_per_person_per_day = data['Unité'] == "Kcal/personne/jour"

# Effectuer le calcul en tenant compte de la population et du nombre de jours
data.loc[is_kcal_per_person_per_day, 'Valeur'] *= data.loc[is_kcal_per_person_per_day, 'Population'] * data.loc[is_kcal_per_person_per_day, 'Nombres de jour']

# Mettre à jour l'unité pour ces lignes
data.loc[is_kcal_per_person_per_day, 'Unité'] = 'Kcal'

In [201]:
data.head()

,Zone,Élément,Produit,Année,Unité,Valeur,Population,Nombres de jour
0,Afghanistan,Production,Blé et produits,2017,kg,4.281000e+09,36296000,365
1,Afghanistan,Importations - Quantité,Blé et produits,2017,kg,2.302000e+09,36296000,365
2,Afghanistan,Variation de stock,Blé et produits,2017,kg,-1.190000e+08,36296000,365
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,kg,0.000000e+00,36296000,365
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,kg,6.701000e+09,36296000,365


In [202]:
data.describe()

,Année,Valeur,Population,Nombres de jour
count,176600.0,1.766000e+05,1.766000e+05,176600.0
mean,2017.0,4.461825e+10,4.606615e+07,365.0
std,0.0,1.890333e+12,1.605016e+08,0.0
min,2017.0,-1.038800e+10,5.200000e+04,365.0
25%,2017.0,0.000000e+00,2.920000e+06,365.0
50%,2017.0,1.229500e+01,1.028800e+07,365.0
75%,2017.0,1.300000e+07,3.195900e+07,365.0
max,2017.0,4.382784e+14,1.421021e+09,365.0


In [203]:
# Pivotez les données pour transformer les unités en colonnes
pivot_data = data.pivot_table(index=['Zone', 'Élément', 'Produit', 'Année'],
                              columns='Unité', values='Valeur', aggfunc='sum').reset_index()
# Afficher les premières lignes du nouveau dataframe
pivot_data.head()

Unité,Zone,Élément,Produit,Année,Kcal,kg
0,Afghanistan,Aliments pour animaux,Arachides Decortiquees,2017,NaN,1000000.0
1,Afghanistan,Aliments pour animaux,Blé et produits,2017,NaN,76000000.0
2,Afghanistan,Aliments pour animaux,Coco (Incl Coprah),2017,NaN,1000000.0
3,Afghanistan,Aliments pour animaux,"Céréales, Autres",2017,NaN,0.0
4,Afghanistan,Aliments pour animaux,Graines Colza/Moutarde,2017,NaN,1000000.0


In [204]:
# Fusionnez le dataframe initial avec le dataframe pivot
data = pd.merge(data, pivot_data, how='left', on=[ 'Zone', 'Élément','Produit', 'Année'])
# Afficher les premières lignes du nouveau dataframe
data.head(10)

,Zone,Élément,Produit,Année,Unité,Valeur,Population,Nombres de jour,Kcal,kg
0,Afghanistan,Production,Blé et produits,2017,kg,4.281000e+09,36296000,365,NaN,4.281000e+09
1,Afghanistan,Importations - Quantité,Blé et produits,2017,kg,2.302000e+09,36296000,365,NaN,2.302000e+09
2,Afghanistan,Variation de stock,Blé et produits,2017,kg,-1.190000e+08,36296000,365,NaN,-1.190000e+08
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,kg,0.000000e+00,36296000,365,NaN,0.000000e+00
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,kg,6.701000e+09,36296000,365,NaN,6.701000e+09
5,Afghanistan,Aliments pour animaux,Blé et produits,2017,kg,7.600000e+07,36296000,365,NaN,7.600000e+07
6,Afghanistan,Semences,Blé et produits,2017,kg,3.440000e+08,36296000,365,NaN,3.440000e+08
7,Afghanistan,Pertes,Blé et produits,2017,kg,6.420000e+08,36296000,365,NaN,6.420000e+08
8,Afghanistan,Résidus,Blé et produits,2017,kg,0.000000e+00,36296000,365,NaN,0.000000e+00
9,Afghanistan,Nourriture,Blé et produits,2017,kg,5.640000e+09,36296000,365,NaN,5.640000e+09


In [205]:
# Liste des colonnes à supprimer
colonnes_a_supprimer = ['Valeur', 'Nombres de jour'], #'Kcal', 'Unité', 'Année']

# Utilisation de la méthode drop pour supprimer les colonnes dans le DataFrame d'origine
data.drop(columns=colonnes_a_supprimer, inplace=True)

# Afficher les premières lignes du nouveau dataframe
data.head()

,Zone,Élément,Produit,Population,kg
0,Afghanistan,Production,Blé et produits,36296000,4.281000e+09
1,Afghanistan,Importations - Quantité,Blé et produits,36296000,2.302000e+09
2,Afghanistan,Variation de stock,Blé et produits,36296000,-1.190000e+08
3,Afghanistan,Exportations - Quantité,Blé et produits,36296000,0.000000e+00
4,Afghanistan,Disponibilité intérieure,Blé et produits,36296000,6.701000e+09


In [206]:
data.describe()

,Population,kg
count,1.766000e+05,1.621240e+05
mean,4.606615e+07,2.281782e+08
std,1.605016e+08,4.970241e+09
min,5.200000e+04,-1.038800e+10
25%,2.920000e+06,0.000000e+00
50%,1.028800e+07,2.340000e+00
75%,3.195900e+07,7.000000e+06
max,1.421021e+09,7.585480e+11


In [207]:
detecter_outliers(data, 'kg', seuil=2.0)

Le pourcentage de valeurs considérées comme des outliers en utilisant le Z-score au seuil 2 dans la colonne 'kg' est 0.29%


In [210]:
# detecter_outliers(data, 'Kcal', seuil=2.0)

In [209]:
detecter_outliers(data, 'Population', seuil=2.0)

Le pourcentage de valeurs considérées comme des outliers en utilisant le Z-score au seuil 2 dans la colonne 'Population' est 1.29%


In [213]:
# Utilisez la fonction pivot_table pour effectuer la transformation
data_pivot = data.pivot_table(index=['Zone'],
                                columns='Élément',
                                values='kg',
                                aggfunc='sum').reset_index()

# Affiche le DataFrame transformé
data_pivot.head()

Élément,Zone,Alimentation pour touristes,Aliments pour animaux,Autres utilisations (non alimentaire),Disponibilité alimentaire (Kcal/personne/jour),Disponibilité alimentaire en quantité (kg/personne/an),Disponibilité de matière grasse en quantité (g/personne/jour),Disponibilité de protéines en quantité (g/personne/jour),Disponibilité intérieure,Exportations - Quantité,Importations - Quantité,Nourriture,Pertes,Production,Résidus,Semences,Traitement,Variation de stock
0,Afghanistan,NaN,4.560000e+08,9.400000e+07,0.0,357.31,4.057875e+08,7.165865e+08,1.513900e+10,6.010000e+08,4.555000e+09,1.296800e+10,1.146000e+09,1.160900e+10,-111000000.0,4.700000e+08,1.150000e+08,426000000.0
1,Afrique du Sud,0.0,9.371000e+09,1.448000e+09,0.0,556.42,1.704615e+09,1.734579e+09,6.684000e+10,1.096800e+10,9.020000e+09,3.172600e+10,3.108000e+09,6.845500e+10,-594000000.0,2.120000e+08,2.156400e+10,-322000000.0
2,Albanie,NaN,6.970000e+08,1.770000e+08,0.0,1260.49,1.237612e+08,1.257929e+08,4.879000e+09,1.560000e+08,8.840000e+08,3.631000e+09,3.210000e+08,4.231000e+09,-115000000.0,5.300000e+07,1.130000e+08,72000000.0
3,Algérie,0.0,4.884000e+09,2.405000e+09,0.0,798.71,1.202818e+09,1.402684e+09,4.536500e+10,7.430000e+08,1.971800e+10,3.305600e+10,3.683000e+09,2.676300e+10,-22000000.0,3.580000e+08,9.930000e+08,374000000.0
4,Allemagne,NaN,3.090400e+10,8.316000e+09,0.0,943.24,4.463978e+09,3.139810e+09,1.749600e+11,5.879300e+10,6.655000e+10,7.798800e+10,5.756000e+09,1.678280e+11,-831000000.0,1.712000e+09,5.110900e+10,651000000.0


## Test

In [63]:
import pandas as pd
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

# Supprimer les colonnes non nécessaires
data_cah = data[['Zone', 'kg', 'Élément']].copy()

# Remplacer les valeurs NaN par 0
data_cah['kg'].fillna(0, inplace=True)

# Pivoter le DataFrame pour avoir les Zones comme index et les colonnes comme variables
data_cah_pivot = data_cah.pivot(index='Zone', columns='Élément', values='kg')

# Remplacer les valeurs NaN par 0
data_cah_pivot.fillna(0, inplace=True)

# Calculer la matrice de liaison
Z = linkage(data_cah_pivot, method='ward')

# Afficher le dendrogramme
plt.figure(figsize=(12, 6))
dendrogram(Z, labels=data_cah_pivot.index)
plt.title('Dendrogramme de la Classification Ascendante Hiérarchique')
plt.xlabel('Zones')
plt.ylabel('Distance Euclidienne')
plt.show()


KeyError: "['kg', 'Élément'] not in index"

In [ ]:
# Importer les bibliothèques nécessaires
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Sélectionner les colonnes pour le K-Means
data_kmeans = data[['Kcal', 'kg']]

# Standardiser les données
scaler = StandardScaler()
data_kmeans_scaled = scaler.fit_transform(data_kmeans)

# Appliquer K-Means avec un nombre de clusters choisi (par exemple, 3 clusters)
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster_KMeans'] = kmeans.fit_predict(data_kmeans_scaled)

c:\Users\antoi\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


## Test normalisation

In [ ]:
# Créez une copie du DataFrame original
data_minmax = data.copy()

# Créez un MinMaxScaler
scaler = MinMaxScaler()

# Colonnes à normaliser
colonnes_a_normaliser = ['kg', 'Kcal']

# Normalisez les colonnes
data_minmax[colonnes_a_normaliser] = scaler.fit_transform(data[colonnes_a_normaliser])

# Afficher les premières lignes du DataFrame
data_minmax.head()

,Zone,Élément,Produit,Année,Unité,Population,Kcal,kg
0,Afghanistan,Production,Blé et produits,2017,kg,36296000,NaN,0.019077
1,Afghanistan,Importations - Quantité,Blé et produits,2017,kg,36296000,NaN,0.016503
2,Afghanistan,Variation de stock,Blé et produits,2017,kg,36296000,NaN,0.013355
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,kg,36296000,NaN,0.013510
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,kg,36296000,NaN,0.022224


In [ ]:
# Créez une copie du DataFrame original
data_standard = data.copy()

# Colonnes à normaliser
numeric_columns = ['Kcal', 'kg']

# Standardiser les données
scaler = StandardScaler()
data_standard[numeric_columns] = scaler.fit_transform(data_standard[numeric_columns])

# Afficher les premières lignes du DataFrame standardisé
data_standard.head()

,Zone,Élément,Produit,Année,Unité,Population,Kcal,kg
0,Afghanistan,Production,Blé et produits,2017,kg,-0.060873,NaN,0.815420
1,Afghanistan,Importations - Quantité,Blé et produits,2017,kg,-0.060873,NaN,0.417249
2,Afghanistan,Variation de stock,Blé et produits,2017,kg,-0.060873,NaN,-0.069852
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,kg,-0.060873,NaN,-0.045909
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,kg,-0.060873,NaN,1.302320


In [ ]:
# # Supprimer les colonnes non numériques (si nécessaire)
# data_for_clustering = data.drop(['Zone', 'Élément', 'Produit', 'Année', 'Unité'], axis=1)